# Quantum Espresso

In [1]:
import os
import subprocess
from ase.build import bulk
from atomistics.workflows.evcurve.helper import (
    analyse_structures_helper as evcurve_analyse_structures,
    generate_structures_helper as evcurve_generate_structures,
)
import matplotlib.pyplot as plt
import pprint
from tqdm import tqdm
from time import sleep

In [2]:
def evaluate_with_quantum_espresso(task_dict, pseudopotentials, kpts):
    from ase.calculators.espresso import Espresso, EspressoProfile
    from atomistics.calculators import evaluate_with_ase
    
    return evaluate_with_ase(
        task_dict=task_dict,
        ase_calculator=Espresso(
            pseudopotentials=pseudopotentials,
            tstress=True,
            tprnfor=True,
            kpts=kpts,
            profile=EspressoProfile(
                 command="flux run pw.x",
                 pseudo_dir="/home/runner/work/executorlib/executorlib/tests/integration",
            ),
        ),
    )["energy"]

In [3]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [4]:
structure_dict = evcurve_generate_structures(
    structure=bulk("Al", a=4.15, cubic=True),
    num_points=7,
    vol_range=0.05,
    axes=("x", "y", "z"),
)

In [5]:
task_loop_dict = {k: {"calc_energy": v} for k, v in structure_dict.items()}

In [6]:
pprint.pp(subprocess.check_output(["flux", "resource", "list"], universal_newlines=True).split("\n"))

['     STATE NNODES   NCORES    NGPUS NODELIST',
 '      free      1        2        1 '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 ' allocated      0        0        0 ',
 '      down      0        0        0 ',
 '']


In [7]:
pprint.pp(subprocess.check_output(["flux", "jobs", "-a"], universal_newlines=True).split("\n"))

['       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO', '']


## FluxClusterExecutor

In [8]:
from executorlib import FluxClusterExecutor

In [9]:
future_dict = {}
with FluxClusterExecutor() as exe:
    for k, v in task_loop_dict.items():
        os.makedirs(os.path.abspath(("strain_%0.2f" % k).replace(".", "_")), exist_ok=True)
        future_dict[k] = exe.submit(
            evaluate_with_quantum_espresso, 
            task_dict=v, 
            pseudopotentials=pseudopotentials, 
            kpts=(3, 3, 3), 
            resource_dict={
                "cores": 1,
                "threads_per_core": 2,
                "cwd": os.path.abspath(("strain_%0.2f" % k).replace(".", "_")),
            },
        )
    sleep(1)
    pprint.pp(subprocess.check_output(["flux", "jobs", "-a"], universal_newlines=True).split("\n"))
    result_dict = {
        k: f.result() 
        for k, f in tqdm(future_dict.items())
    }
    sleep(1)
    pprint.pp(subprocess.check_output(["flux", "jobs", "-a"], universal_newlines=True).split("\n"))

['       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO',
 '    ƒEmWEBP5 jan      2025-03-2+  S      2      -        - ',
 '    ƒErvXZ8B jan      2025-03-2+  S      2      -        - ',
 '    ƒEcB6hXD jan      2025-03-2+  R      2      1   0.846s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '']


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:24<00:00, 29.15s/it]


['       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO',
 '    ƒFSHFPtB jan      2025-03-2+ CD      2      1   9.355s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒFLoz3aP jan      2025-03-2+ CD      2      1   26.28s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒFF5tpT9 jan      2025-03-2+ CD      2      1   27.94s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒF2grpmu jan      2025-03-2+ CD      2      1    24.2s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒErvXZ8B jan      2025-03-2+ CD      2      1   43.07s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒEmWEBP5 jan      2025-03-2+ CD      2      1   28.25s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒEcB6hXD jan      2025-03-2+ CD      2      1   46.94s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '']


In [10]:
fit_dict = evcurve_analyse_structures(
    output_dict={"energy": result_dict},
    structure_dict=structure_dict,
    fit_type="polynomial",
    fit_order=3,
)

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
/tmp/ipykernel_31059/4024930470.py:2: SyntaxWarning: invalid escape sequence '\A'
  plt.xlabel("Volume [$\AA^3$]")


In [11]:
plt.plot(fit_dict["volume"], fit_dict["energy"], label="$B_0$= %0.2f GPa" % fit_dict["bulkmodul_eq"])
plt.xlabel("Volume [$\AA^3$]")
plt.ylabel("Energy [eV]")
plt.legend()

## FluxJobExecutor

In [12]:
from executorlib import FluxJobExecutor

In [13]:
future_dict = {}
with FluxJobExecutor(flux_executor_nesting=True) as exe:
    for k, v in task_loop_dict.items():
        os.makedirs(os.path.abspath(("strain_%0.2f" % k).replace(".", "_")), exist_ok=True)
        future_dict[k] = exe.submit(
            evaluate_with_quantum_espresso, 
            task_dict=v, 
            pseudopotentials=pseudopotentials, 
            kpts=(3, 3, 3), 
            resource_dict={
                "cores": 1, 
                "threads_per_core": 2, 
                "cwd": os.path.abspath(("strain_%0.2f" % k).replace(".", "_")),
            },
        )
    sleep(1)
    pprint.pp(subprocess.check_output(["flux", "jobs", "-a"], universal_newlines=True).split("\n"))
    result_dict = {
        k: f.result() 
        for k, f in tqdm(future_dict.items())
    }
    sleep(1)
    pprint.pp(subprocess.check_output(["flux", "jobs", "-a"], universal_newlines=True).split("\n"))

['       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO',
 '   ƒ2onx4eSU jan      flux        S      1      -        - ',
 '   ƒ2onyYdio jan      flux        S      1      -        - ',
 '   ƒ2onyYdip jan      flux        S      1      -        - ',
 '   ƒ2oo12d19 jan      flux        S      1      -        - ',
 '   ƒ2oo12d1A jan      flux        S      1      -        - ',
 '   ƒ2oo12d1B jan      flux        S      1      -        - ',
 '   ƒ2onx4eST jan      flux        R      1      1   0.994s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒFSHFPtB jan      2025-03-2+ CD      2      1   9.355s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒFLoz3aP jan      2025-03-2+ CD      2      1   26.28s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒFF5tpT9 jan      2025-03-2+ CD      2      1   27.94s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒF2grpmu jan      2025-03-2+ CD      2      1    24.2s '
 

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:10<00:00, 27.22s/it]


['       JOBID USER     NAME       ST NTASKS NNODES     TIME INFO',
 '   ƒ2oo12d1B jan      flux       CD      1      1   26.04s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '   ƒ2oo12d1A jan      flux       CD      1      1   43.78s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '   ƒ2oo12d19 jan      flux       CD      1      1   21.92s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '   ƒ2onyYdip jan      flux       CD      1      1   8.466s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '   ƒ2onyYdio jan      flux       CD      1      1   23.27s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '   ƒ2onx4eSU jan      flux       CD      1      1   43.42s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '   ƒ2onx4eST jan      flux       CD      1      1   25.73s '
 'p200300e77f488c5f903797efb75a6bf7.dip0.t-ipconnect.de',
 '    ƒFSHFPtB jan      2025-03-2+ CD      2      1   9.355s '
 'p200300e77f488c5f903

In [14]:
fit_dict = evcurve_analyse_structures(
    output_dict={"energy": result_dict},
    structure_dict=structure_dict,
    fit_type="polynomial",
    fit_order=3,
)

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
/tmp/ipykernel_31059/4024930470.py:2: SyntaxWarning: invalid escape sequence '\A'
  plt.xlabel("Volume [$\AA^3$]")


In [15]:
plt.plot(fit_dict["volume"], fit_dict["energy"], label="$B_0$= %0.2f GPa" % fit_dict["bulkmodul_eq"])
plt.xlabel("Volume [$\AA^3$]")
plt.ylabel("Energy [eV]")
plt.legend()